In [5]:
# Declare Dependencies 
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import os
import time
import requests
import warnings
import pymongo
warnings.filterwarnings('ignore')

In [ ]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

In [6]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', executable_path, headless=False)

/usr/local/bin/chromedriver


In [7]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [9]:
html = browser.html
soup = bs(html, 'html.parser')

In [10]:
# get the news title and paragraph text
news_title= soup.find('div', class_= 'content_title').text
news_title

'Mars Now'

In [11]:
paragraph_text = soup.find('div', class_='article_teaser_body').text
paragraph_text

"A crisply rendered web application can show you where the agency's Mars 2020 mission is right now as it makes its way to the Red Planet for a Feb. 18, 2021, landing."

In [13]:
# get the featured image
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(featured_image_url)

In [15]:
html_image = browser.html
soup = bs(html_image, 'html.parser')

In [16]:
featured_image_url  = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]
main_url = 'https://www.jpl.nasa.gov'
featured_image_url = main_url + featured_image_url
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16711-1920x1200.jpg'

In [17]:
# get Mars Facts, get the table from the site, convert to html
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)

In [18]:
html = browser.html
soup = bs(html, 'html.parser')

In [21]:
tables = pd.read_html(facts_url)
mars_df = tables[1]
mars_df.head()

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days


In [25]:
mars_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars - Earth Comparison</th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Moons:</td>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Temperature:</td>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

In [35]:
# Mars Hemispheres
hemisphere_url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)

In [36]:
html = browser.html
soup = bs(html, 'html.parser')

In [37]:
# loop through the urls with bs and scrape titles and image urls
items = soup.find_all('div', class_='item')
hem_image_urls = []
hem_main_url = 'https://astrogeology.usgs.gov'

In [38]:
# loop
for item in items: 
    title = item.find('h3').text
    image_url = item.find('a', class_='itemLink product-item')['href']
    browser.visit(hem_main_url + image_url)
    image_html = browser.html
    soup = bs(image_html, 'html.parser')
    image_url = hem_main_url + soup.find('img', class_='wide-image')['src']
    hem_image_urls.append({"Title" : title, "Image_URL" : image_url})
hem_image_urls


[{'Title': 'Cerberus Hemisphere Enhanced',
  'Image_URL': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'Title': 'Schiaparelli Hemisphere Enhanced',
  'Image_URL': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'Title': 'Syrtis Major Hemisphere Enhanced',
  'Image_URL': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'Title': 'Valles Marineris Hemisphere Enhanced',
  'Image_URL': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]